# **Лабораторна робота No2**
**Наука про дані: підготовчий етап**

**Мета роботи:** ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі та природу даних, над якими виконується аналітичні операції.

# **Хід виконання роботи:**

1. Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження. Передбачити повторні запуски скрипту, довантаження нових даних та колізію даних;

In [8]:
import urllib.request 
import datetime 
import time 
import os 
import pandas as pd

if not os.path.isdir("data"): 
    os.mkdir("data") 

for i in range(1,28): 
    url= f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean" 
    wp = urllib.request.urlopen(url)
    text = wp.read()
    now = datetime.datetime.now() 
    date_and_time = now.strftime("%d-%m-%Y__%H-%M-%S")
   
    with open(f'data/id_{str(i)}_{date_and_time}.csv', 'wb') as f: 
        f.write(text) 
    print(f"Файл {i} області збережено") 

print("Всі файли збережено")

# Зчитування CSV-файлів у фрейм
def read_csv_files_to_df(directory):
    data_frames = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            reg = filename.split("_")[1]
            file_path = os.path.join(directory, filename)
            try:
                data = pd.read_csv(file_path, index_col=False, header=1)
                data.columns = [col.strip().lower().replace("<br>", "") for col in data.columns]
                data = data.replace(to_replace=r'<.*?>', value='', regex=True)
                data["oblast"] = reg
                data_frames.append(data)
            except Exception as e:
                print(f"Не вдалося зчитати файл {file_path}: {e}")
    if data_frames:
        combined_data = pd.concat(data_frames, ignore_index=True)
    else:
        combined_data = pd.DataFrame()
    combined_data = combined_data[combined_data["vhi"] != -1]
    return combined_data

# Словник заміни ID на назви областей
obl_dict = {
    1: "Черкаська", 2: "Чернігівська", 3: "Чернівецька", 4: "Республіка Крим",
    5: "Дніпропетровська", 6: "Донецька", 7: "Івано-Франківська", 8: "Харківська",
    9: "Херсонська", 10: "Хмельницька", 11: "Київська", 13: "Кіровоградська",
    14: "Луганська", 15: "Львівська", 16: "Миколаївська", 17: "Одеська",
    18: "Полтавська", 19: "Рівенська", 21: "Сумська", 22: "Тернопільська",
    23: "Закарпатська", 24: "Вінницька", 25: "Волинська", 26: "Запорізька",
    27: "Житомирська"
}

# Зчитування та об'єднання
data = read_csv_files_to_df("data")
data["oblast"] = data["oblast"].astype(int).map(obl_dict)
data["year"] = pd.to_numeric(data["year"], errors="coerce").astype("Int64")

# Збереження у фінальний об'єднаний файл
data.to_csv("data/all_regions_combined.csv", index=False, encoding='utf-8')
print("Об’єднаний файл збережено у 'data/all_regions_combined.csv'")


Файл 1 області збережено
Файл 2 області збережено
Файл 3 області збережено
Файл 4 області збережено
Файл 5 області збережено
Файл 6 області збережено
Файл 7 області збережено
Файл 8 області збережено
Файл 9 області збережено
Файл 10 області збережено
Файл 11 області збережено
Файл 12 області збережено
Файл 13 області збережено
Файл 14 області збережено
Файл 15 області збережено
Файл 16 області збережено
Файл 17 області збережено
Файл 18 області збережено
Файл 19 області збережено
Файл 20 області збережено
Файл 21 області збережено
Файл 22 області збережено
Файл 23 області збережено
Файл 24 області збережено
Файл 25 області збережено
Файл 26 області збережено
Файл 27 області збережено
Всі файли збережено
Об’єднаний файл збережено у 'data/all_regions_combined.csv'


In [9]:
print(data.head()) 

   year  week    smn     smt    vci    tci    vhi       oblast
0  1982   1.0  0.059  258.24  51.11  48.78  49.95  Хмельницька
1  1982   2.0  0.063  261.53  55.89  38.20  47.04  Хмельницька
2  1982   3.0  0.063  263.45  57.30  32.69  44.99  Хмельницька
3  1982   4.0  0.061  265.10  53.96  28.62  41.29  Хмельницька
4  1982   5.0  0.058  266.42  46.87  28.57  37.72  Хмельницька


2. Зчитати завантажені текстові файли у фрейм. Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [1]:
import os
import pandas as pd

def read_csv_files_to_df(directory):
    data_frames = []

    for filename in os.listdir(directory):
        reg = filename.split("_")[1]
        file_path = os.path.join(directory, filename)

        try:
            data = pd.read_csv(file_path, index_col=False, header=1)
            data.columns = [col.strip().lower().replace("<br>", "") for col in data.columns] 
            data = data.replace(to_replace=r'<.*?>', value='', regex=True)
            data["oblast"] = reg
            data_frames.append(data)

        except Exception as e:
            print(f"Не вдалося зчитати файл {file_path}: {e}")


    if data_frames:
        combined_data = pd.concat(data_frames, ignore_index=True)
    else:
        combined_data = pd.Dataframe()
    
    combined_data = combined_data[combined_data["vhi"] != -1]
    return combined_data


data = read_csv_files_to_df("data") 
print(data.head())   

   year  week    smn     smt    vci    tci    vhi oblast
0  1982   1.0  0.059  258.24  51.11  48.78  49.95     10
1  1982   2.0  0.063  261.53  55.89  38.20  47.04     10
2  1982   3.0  0.063  263.45  57.30  32.69  44.99     10
3  1982   4.0  0.061  265.10  53.96  28.62  41.29     10
4  1982   5.0  0.058  266.42  46.87  28.57  37.72     10


3. Реалізувати окрему процедуру, яка змінить індекси областей, які використані на порталі NOAA (за англійською абеткою) на наступні, за українською (виключно старі індекси на нові)

In [2]:
#obl_dict={ 
#1: 22,   # Cherkasy -> Черкаська
#2: 24,   # Chernihiv -> Чернігівська
#3: 23,   # Chernivtsi -> Чернівецька
#4: 25,   # Crimea -> Республіка Крим
#5: 3,    # Dnipropetrovs'k -> Дніпропетровська
#6: 4,    # Donets'k -> Донецька
#7: 8,    # Ivano-Frankivs'k -> Івано-Франківська
#8: 19,   # Kharkiv -> Харківська
#9: 20,   # Kherson -> Херсонська
#10: 21,  # Khmel'nyts'kyy -> Хмельницька
#11: 9,   # Kiev -> Київська
#12: 9,   # Kiev City -> Київська
#13: 10,  # Kirovohrad -> Кіровоградська
#14: 11,  # Luhans'k -> Луганська
#15: 12,  # L'viv -> Львівська#
#16: 13,  # Mykolayiv -> Миколаївська
#17: 14,  # Odessa -> Одеська
#18: 15,  # Poltava -> Полтавська
#19: 16,  # Rivne -> Рівенська
#20: 25,  # Sevastopol -> Республіка Крим
#21: 17,  # Sumy -> Сумська
#22: 18,  # Ternopil' -> Тернопільська
#23: 6,   # Transcarpathia -> Закарпатська
#24: 1,   # Vinnytsya -> Вінницька
#25: 2,   # Volyn -> Волинська
#26: 7,   # Zaporizhzhya -> Запорізька
#27: 5    # Zhytomyr -> Житомирська
#} 

obl_dict={ 
1: "Черкаська",   # Cherkasy -> Черкаська
2: "Чернігівська",   # Chernihiv -> Чернігівська
3: "Чернівецька",   # Chernivtsi -> Чернівецька
4: "Республіка Крим",   # Crimea -> Республіка Крим
5: "Дніпропетровська",    # Dnipropetrovs'k -> Дніпропетровська
6: "Донецька",    # Donets'k -> Донецька
7: "Івано-Франківська",    # Ivano-Frankivs'k -> Івано-Франківська
8: "Харківська",   # Kharkiv -> Харківська
9: "Херсонська",   # Kherson -> Херсонська
10: "Хмельницька",  # Khmel'nyts'kyy -> Хмельницька
11: "Київська",   # Kiev -> Київська
13: "Кіровоградська",  # Kirovohrad -> Кіровоградська
14: "Луганська",  # Luhans'k -> Луганська
15: "Львівська",  # L'viv -> Львівська
16: "Миколаївська",  # Mykolayiv -> Миколаївська
17: "Одеська",  # Odessa -> Одеська
18: "Полтавська",  # Poltava -> Полтавська
19: "Рівенська",  # Rivne -> Рівенська
21: "Сумська",  # Sumy -> Сумська
22: "Тернопільська",  # Ternopil' -> Тернопільська
23: "Закарпатська",   # Transcarpathia -> Закарпатська
24: "Вінницька",   # Vinnytsya -> Вінницька
25: "Волинська",   # Volyn -> Волинська
26: "Запорізька",   # Zaporizhzhya -> Запорізька
27: "Житомирська"    # Zhytomyr -> Житомирська
} 

data["oblast"] = data["oblast"].astype(int).map(obl_dict)


In [3]:
data["year"] = pd.to_numeric(data["year"], errors="coerce").astype("Int64")
print(data.head())

   year  week    smn     smt    vci    tci    vhi       oblast
0  1982   1.0  0.059  258.24  51.11  48.78  49.95  Хмельницька
1  1982   2.0  0.063  261.53  55.89  38.20  47.04  Хмельницька
2  1982   3.0  0.063  263.45  57.30  32.69  44.99  Хмельницька
3  1982   4.0  0.061  265.10  53.96  28.62  41.29  Хмельницька
4  1982   5.0  0.058  266.42  46.87  28.57  37.72  Хмельницька


4. Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
1) Ряд VHI для області за вказаний рік;
2) Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани;
3) Ряд VHI за вказаний діапазон років для вказаних областей;
4) Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25). Повернути роки, назви областей з екстремальними посухами;

In [5]:
import statistics 

def vhi_obl_year(obl, year): 
    res = data[(data["oblast"] == obl) & (data["year"] == year)] 
    res_vhi = list(res["vhi"]) 
    
    print(f"vhi для {obl} області на {year} рік склав \n{res_vhi}") 
    
    print("\n\tMin:", min(res_vhi)) 
    print("\tMax:", max(res_vhi)) 
    
    print("\n\tAvg:", statistics.mean(res_vhi)) 
    print("\tMedian:", statistics.median(res_vhi)) 

vhi_obl_year(22, 2003)

vhi для 22 області на 2003 рік склав 
[49.76, 45.72, 44.91, 45.02, 43.94, 43.97, 45.03, 46.16, 47.66, 46.93, 46.43, 46.14, 45.01, 45.01, 43.53, 40.05, 34.14, 29.44, 28.14, 28.62, 30.27, 33.34, 38.78, 44.66, 50.44, 54.97, 58.9, 62.22, 63.94, 63.65, 60.95, 57.16, 56.58, 55.19, 51.23, 49.8, 48.86, 49.72, 52.6, 54.33, 53.72, 50.44, 43.07, 35.69, 33.71, 37.14, 41.48, 46.87, 53.06, 60.32, 67.03]

	Min: 28.14
	Max: 67.03

	Avg: 47.171176470588236
	Median: 46.43


In [6]:
def vhi_obls_years(obl, min_year, max_year): 
    res = data[(data["oblast"].isin(obl)) & (data["year"].between(min_year, max_year+1))] 
    res_vhi = list(res["vhi"]) 
    
    print(f"vhi для {obl} області на {min_year}-{max_year} має {len(res_vhi)} значень") 
    
    print("\n\tMin:", min(res_vhi)) 
    print("\tMax:", max(res_vhi)) 
    
    print("\n\tAvg:", statistics.mean(res_vhi)) 
    print("\tMedian:", statistics.median(res_vhi)) 
vhi_obls_years([22, 2, 10], 1990, 2021)

vhi для [22, 2, 10] області на 1990-2021 має 5031 значень

	Min: 10.68
	Max: 84.52

	Avg: 48.759980123235934
	Median: 47.95


In [7]:
def drought_years(percent=0):
    draught_years = []
    oblasti_indices = data["oblast"].unique()  
    min_obl_num = int((percent / 100) * len(oblasti_indices))  
    
    for year in data["year"].unique():
        draughts_in = 0
        for idx in oblasti_indices:
            res = data[(data["oblast"] == idx) & (data["year"] == year)]
            res_vhi = list(res["vhi"])
            
            if len(res_vhi) == 0:
                continue  
                
            if statistics.mean(res_vhi) < 35:
                draughts_in += 1
                
        if draughts_in > min_obl_num:
            draught_years.append(int(year))
    
    print(f"Роки, коли посухи торкнулися >{percent}% областей ({min_obl_num}+ областей):")
    print(draught_years)

drought_years(20)

Роки, коли посухи торкнулися >20% областей (5+ областей):
[1986]
